In [1]:
%pip install pandas transformers tokenizers datasets xformers
%pip install zstandard jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.9 MB/s eta 

In [2]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json, os
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

device = torch.device("cuda")
os.environ['CURL_CA_BUNDLE'] = ''
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")

# import utils

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/ru_turbo_alpaca.jsonl', encoding = 'utf-8') as f:
  data_alpaca = [json.loads(line) for line in f]

In [5]:
attributes_to_keep = ["instruction", "input", "output"]
rus_alpaca = [{k: v for k, v in example.items() if k in attributes_to_keep} for example in data_alpaca]
rus_alpaca[:10]

[{'instruction': 'Опишите, как сделать горшок из глины.',
  'input': '',
  'output': 'Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.'},
 {'instruction': 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'input': '',
  'output': 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.'},
 {'instruction': 'Напиши аналог данной пословицы на 

In [6]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведена инструкция, описывающая задачу, инструкции соответствует input, который приводит дополнительный контекст. "
        "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Ниже приведена инструкция, описывающая задание. "
        "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [7]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [8]:
# !pip install accelerate -U

In [9]:
# !pip install transformers[torch]

In [10]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [11]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        #list_data_dict = json.load(open(data_path))
        list_data_dict = rus_alpaca

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [12]:
model_name = "ai-forever/rugpt3small_based_on_gpt2"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [13]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="/ru_turbo_alpaca.jsonl")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [14]:
train_args = transformers.TrainingArguments(learning_rate=1e-5,
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="rugpt3small_instruct_ft")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [16]:
trainer.train()

Step,Training Loss
1000,2.555200
2000,2.234700
3000,2.185000
4000,2.167800
5000,2.143600
6000,2.090400
7000,2.102600
8000,2.091900
9000,2.069500
10000,2.062200


TrainOutput(global_step=14911, training_loss=2.1312332813364505, metrics={'train_runtime': 2121.3469, 'train_samples_per_second': 14.058, 'train_steps_per_second': 7.029, 'total_flos': 2198460123648000.0, 'train_loss': 2.1312332813364505, 'epoch': 1.0})

In [17]:
trainer.save_model('rugpt3small_02')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODEL_NAME = 'rugpt3small_02'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [19]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Ниже приведена инструкция, описывающая задачу, инструкции соответствует input, который приводит дополнительный контекст. "
              "Напишите ответ, который соответствующим образом выполняет запрос.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [20]:
text = """
Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов, что привело к длительным отменам по крайней мере на двух линиях.
Во вторник днем все поезда были остановлены на оживленной линии между южными городами Ден Босх и Бокстел после того, как животные зарылись в дамбу, несущую рельсы. Национальная железнодорожная компания сообщила, что линия не будет работать как минимум неделю.
Подкоп означает, что "рельсы могут проседать, и тогда безопасность движения поездов больше не гарантируется", - говорится в заявлении ProRail, компании, обслуживающей железнодорожную сеть Нидерландов.
Ранее в этом месяце барсуки также зарылись под рельсы в районе северной деревни Молкверум в провинции Фрисландия, что вывело линию из эксплуатации до следующего месяца, пока рабочие будут добиваться разрешения на перемещение животных.
Барсуки являются охраняемыми животными в Нидерландах, поэтому железнодорожники должны получить разрешение на их перемещение или нарушение их среды обитания, прежде чем начать ремонт.
"""

In [21]:
instruction = "Дайте краткое изложение этого текста."
predict_for_instruction(instruction, text, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов,'

In [22]:
instruction = "Give a very short summary of this text."
predict_for_instruction(instruction, text, model)

'Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов,'

In [23]:
instruction = "Напишите заголовок для следующего текста."
predict_for_instruction(instruction, text, model)

'Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов,'

In [27]:
text3 = ''
instruction = "Назови шесть фруктов и овощей"
predict_for_instruction(instruction, text3, model)

'1. \\"Малина\\"\n2. \\"Сладкая клубника\\"'

In [28]:
instruction = "Напиши рецепт омлета"
predict_for_instruction(instruction, text3, model)

'Ингредиенты:\n- 2 яйца\n- 1 стакан молока\n- 1 стакан сахара\n-'